# Tides acting on a post- natal kick binary

Study of the impact of tides on non-coplanar binaries

In this notebook we show the results of our code for the evolution of tides acting on a HMXB using different values for the inclination just after a momentum kick

### load modules

In [ ]:
# auto load modules, no need to restart kernel
%load_ext autoreload
%autoreload 2

from matplotlib.pyplot import cm
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import MultipleLocator

# custom module to read MESA output
from mesa_reader import *

### utility functions

#### 1. group_consecutives

Group lists by consecutive numbers. Useful when having several mass-transfer (MT) phases

In [ ]:
def group_consecutives(vals, step=1):
    '''Return list of consecutive lists of numbers from vals (number list)
    '''
    run = []
    result = [run]
    expect = None
    for v in vals:
        if (v == expect) or (expect is None):
            run.append(int(v))
        else:
            run = [int(v)]
            result.append(run)
        expect = int(v) + step
    return result

def find_tams_age(center_h1, age):
    '''Find the value of the terminal age main sequence (TAMS)'''
    for k, h1 in enumerate(center_h1):
        if h1 < 1e-4:
            return age[k]

---

### inclinations used

In [ ]:
inclinations = (0, 20, 40, 60, 80, 100, 120, 140, 160)

# hex color from https://coolors.co/palettes/popular/gradient
colorbar = (
    '#000000', '#012A4A', '#013A63', '#01497C', '#014F86', '#2A6F97',
    '#2C7DA0', '#468FAF', '#61A5C2', '#89C2D9', '#A9D6E5')

### load data

In [ ]:
root = '../data/asb2022/caseB'

# dictionaries with MESA output plus the location of the TAMS for the star
t_tams = dict()
binary, companion = dict(), dict()

for inclination in inclinations:
    companion[inclination] = MesaInfo(f'{root}/{inclination}/LOGS_{inclination}/history.data')
    binary[inclination] = MesaInfo(f'{root}/{inclination}/binary_history_{inclination}.data')
    t_tams[inclination] = find_tams_age(companion[inclination].data.center_h1, companion[inclination].data.star_age)

In [ ]:
# find when there is a first MT phase in the evolution of the binary
i = 0 # inclination
rl_gap = (binary[i].data.star_1_radius - binary[i].data.rl_1) \
          / binary[i].data.rl_1
mask = (rl_gap  > 0)
init_MT = binary[i].data.age[mask][0] - t_tams[i]

## plots

### evolution of orbital parameters

### - eccentricity and orbital periods

In [ ]:
plt.style.use('style.mpl')
fig, axs = plt.subplots(figsize=(4,4), nrows=2, sharex=True)
plt.subplots_adjust(hspace=0)

left, bottom, width, height = [0.22, 0.68, 0.3, 0.17]
ax2 = fig.add_axes([left, bottom, width, height])

# tick labels
latex = '{\\rm TAMS}'
axs[-1].set_xlabel(f'$t - t_{latex}$')
# first, orbital period
latex = '{\\rm orb}'
axs[0].set_ylabel(f'$P_{latex}$ [days]')
# follows the eccentricity
axs[1].set_ylabel('eccentricity')

for k, inclination in enumerate(inclinations):
    data_null = binary[inclination]
    tams_age = t_tams[inclination]
    
    axs[0].plot(
        data_null.data.age-tams_age, data_null.data.period_days,
        color=colorbar[k]
    );
    
    ax2.plot(
        data_null.data.age-tams_age, data_null.data.period_days,
        color=colorbar[k]
    );
    
    axs[1].plot(
        data_null.data.age-tams_age, data_null.data.eccentricity,
        color=colorbar[k],
        label=f'{inclination} deg'
    );

# begin of MT phase
axs[0].axvline(x=init_MT, color='dimgray', ls=':')
axs[1].axvline(x=init_MT, color='dimgray', ls=':')
axs[0].annotate(
    'MT phase', xy=(9e3,90),
    bbox=dict(
        facecolor='lightgray', edgecolor='dimgray',
        boxstyle='round, pad=0.3'
    )
);

# legend
leg = axs[1].legend(
    bbox_to_anchor=(0.72,0.2),
    title='inclinations', fontsize=6, title_fontsize=8
);
for line, text in zip(leg.get_lines(), leg.get_texts()):
    text.set_color(line.get_color())

# ticks params & limits
axs[0].set_yticks(np.arange(40, 120, 20)); axs[0].set_ylim([45, 105])
axs[0].yaxis.set_minor_locator(MultipleLocator(10))

axs[1].set_yticks(np.arange(0, 1.2, 0.1)); axs[1].set_ylim([0, 0.33])
axs[1].yaxis.set_minor_locator(MultipleLocator(0.05))

axs[-1].set_xticks(np.arange(6e3, 1.2e4, 1e3)); axs[-1].set_xlim([6e3, 1.1e4])
axs[-1].xaxis.set_minor_locator(MultipleLocator(200))

ax2.set_xticks(np.arange(6e3, 1.2e4, 1e3)); ax2.set_xlim([6.8e3, 9.2e3])
ax2.xaxis.set_minor_locator(MultipleLocator(200))

ax2.set_yticks(np.arange(50, 60, 2)); ax2.set_ylim([51, 57])
ax2.yaxis.set_minor_locator(MultipleLocator(1))

# reduce fontsize for inset plot
for item in ([ax2.title, ax2.xaxis.label, ax2.yaxis.label] +
             ax2.get_xticklabels() + ax2.get_yticklabels()):
    item.set_fontsize(6);

### - inclination and spin period of each star

In [ ]:
plt.style.use('style.mpl')
fig, axs = plt.subplots(figsize=(4,4), nrows=2, sharex=True)
plt.subplots_adjust(hspace=0)

# tick labels
latex = '{\\rm TAMS}'
axs[-1].set_xlabel(f'$t/t_{latex}$')
# first, orbital period
latex = '{\\rm orb}'
axs[0].set_ylabel('inclination [deg]')
# follows the rot & orb periods
latex_rot = '{\\rm rot}'
latex_orb = '{\\rm orb}'
axs[1].set_ylabel(f'$P_{latex_rot}/P_{latex_orb}$')

for k, inclination in enumerate(inclinations):
    data_null = binary[inclination]
    tams_age = t_tams[inclination]
    
    axs[0].plot(
        data_null.data.age-tams_age, data_null.data.inclination_1,
        color=colorbar[k],
        label=f'{inclination} deg'
    );

    axs[1].plot(
        data_null.data.age-tams_age, data_null.data.P_rot_div_P_orb_1,
        color=colorbar[k],
        label=f'{inclination} deg'
    );

# ticks params & limits
axs[0].set_yticks(np.arange(0, 181, 45)); axs[0].set_ylim([-10, 180])
axs[0].yaxis.set_minor_locator(MultipleLocator(45/2))

axs[1].set_yticks(np.arange(0, 2, 0.2)); axs[1].set_ylim([0.7, 1.3])
axs[1].yaxis.set_minor_locator(MultipleLocator(0.05))

axs[-1].set_xticks(np.arange(6e3, 1.2e4, 1e3)); axs[-1].set_xlim([6e3, 1.1e4])
axs[-1].xaxis.set_minor_locator(MultipleLocator(200))
    
# begin of MT phase
axs[0].axvline(x=init_MT, color='dimgray', ls=':')
axs[1].axvline(x=init_MT, color='dimgray', ls=':')
axs[0].annotate(
    'MT phase', xy=(9e3,150),
    bbox=dict(
        facecolor='lightgray', edgecolor='dimgray',
        boxstyle='round, pad=0.3'
    )
);

# legend
leg = axs[1].legend(
    bbox_to_anchor=(0.72,0.76),
    title='inclinations', fontsize=6, title_fontsize=8
);
for line, text in zip(leg.get_lines(), leg.get_texts()):
    text.set_color(line.get_color()) 